In [25]:
from functools import total_ordering

# Lab 6: Object-Oriented Python (Part 2!)

## Overview

Congrats on making it to Part 2 of this lab - you're a rockstar!!

After the past two weeks of lecture, which mostly covered rules, definitions, and semantics of classes, we'll be playing around with actual classes today, writing a fair amount of code and building several classes to model a variety of objects.

Recall our starting definitions:

- An *object* has identity
- A *name* is a reference to an object
- A *namespace* is an associative mapping from names to objects
- An *attribute* is any name following a dot ('.')

## Stanford Courses

### Basic Class

Let’s create a class to represent courses at Stanford! A course will have three attributes to start: a department (like `"CS"`), a course code (like `"41"` or `"92SI"`), and a title (like `"hap.py code"`).

```Python
class StanfordCourse:
    def __init__(self, department, code, title):
        self.department = department
        self.code = code
        self.title = title
```

You can assume that all arguments to this constructor will be strings.

Running the following code cell will create a class object `StanfordCourse` and print some information about it.

*Note: If you change the content of this class definition, you will need to re-execute the code cell for it to have any effect. Any instance objects of the old class object will not be automatically updated, so you may need to rerun instantiations of this class object as well.*

In [2]:
class StanfordCourse:
    def __init__(self, department, code, title):
        self.department = department
        self.code = code
        self.title = title
        
print(StanfordCourse)
print(StanfordCourse.mro())
print(StanfordCourse.__init__)

<class '__main__.StanfordCourse'>
[<class '__main__.StanfordCourse'>, <class 'object'>]
<function StanfordCourse.__init__ at 0x7f86d0ed3e60>


We create an instance of the class by instantiating the class object, supplying some arguments.

```Python
stanford_python = StanfordCourse("CS", "41", "hap.py code: the python programming language")
```

Print out the three attributes of the `stanford_python` instance object.

In [3]:
stanford_python = StanfordCourse("CS", "41", "hap.py code: the python programming language")

print(stanford_python.department)  # Print out the department of stanford_python
print(stanford_python.code)  # Print out the code of stanford_python
print(stanford_python.title)  # Print out the title of stanford_python

CS
41
hap.py code: the python programming language


### Inheritance

Let's explore inheritance by creating a `StanfordCSCourse` class that takes an additional parameter `recorded` that defaults to `False`.

In [4]:
class StanfordCSCourse(StanfordCourse):
    def __init__(self, department, code, title, recorded=False):
        super().__init__(department, code, title)
        self.is_recorded = recorded

We haven't seen the `super()` call yet, and it's mostly just magic, but it concretely lets us treat the `self` object as an instance object of the immediate superclass (as measured by MRO), so we can call the superclass's `__init__` method.

We can instantiate our new class:

```Python
a = StanfordCourse("CS", "106A", "Programming Methodology")
b = StanfordCSCourse("CS", "106B", "Programming Abstractions")
x = StanfordCSCourse("CS", "106X", "Programming Abstractions", recorded=True)
print(a.code)  # => "106A"
print(b.code)  # => "106B"
```

Read through the following statements and try to predict their output.

```Python
type(a)
isinstance(a, StanfordCourse)
isinstance(b, StanfordCourse)
isinstance(x, StanfordCourse)
isinstance(x, StanfordCSCourse)
issubclass(x, StanfordCSCourse)
issubclass(StanfordCourse, StanfordCSCourse)
type(a) == type(b)
type(b) == type(x)
a == b
b == x
```

In [6]:
a = StanfordCourse("CS", "106A", "Programming Methodology")
b = StanfordCSCourse("CS", "106B", "Programming Abstractions")
x = StanfordCSCourse("CS", "106X", "Programming Abstractions", recorded=True)

print(type(a)) # class StanfordCourse
print(isinstance(a, StanfordCourse)) # True
print(isinstance(b, StanfordCourse)) # True
print(isinstance(x, StanfordCourse)) # True
print(isinstance(x, StanfordCSCourse)) # True
print(issubclass(StanfordCourse, StanfordCSCourse)) # False
print(issubclass(StanfordCSCourse, StanfordCourse)) # True
print(type(a) == type(b)) # False
print(type(b) == type(x)) # True
print(a == b) # False
print(b == x) # False

<class '__main__.StanfordCourse'>
True
True
True
True
False
True
False
True
False
False


### Additional Attributes

Let's add more functionality to the `StanfordCourse` class!

* Add a attribute `students` to the instances of the `StanfordCourse` class that tracks whether students are present. Initially, students should be an empty set.
* Create a method `mark_attendance(*students)` that takes a variadic number of `students` and marks them as present.
* Create a method `is_present(student)` that takes a student’s name as a parameter and returns `True` if the student is present and `False` otherwise.

In [18]:
class StanfordCourse:
    def __init__(self, department, code, title):
        self.department = department
        self.code = code
        self.title = title
        self.students = set()
        
    def mark_attendance(self, *students):
        # set.update() takes an iterable as input, and evaluates each argument in iterable as a set first. Thus, a string will actually get broken down into individual letters
        for student in students:
            self.students.add(student)
    
    def is_present(self, student):
        return student in self.students


In [20]:
pe = StanfordCourse('PE', '31', 'Advanced Physical Education')
print(pe.students)
attended_students = ['Sam', 'Brian', 'Monica']
pe.mark_attendance(*attended_students)
print(pe.students)
print(pe.is_present('Sue'))
print(pe.is_present('Sam'))

set()
{'Brian', 'Sam', 'Monica'}
False
True


### Implementing Prerequisites

Now, we'll focus on `StanfordCSCourse`. We want to implement functionality to determine if one computer science course is a prerequisite of another. In our implementation, we will assume that the ordering for courses is determined first by the numeric part of the course code: for example, `140` comes before `255`. If there is a tie, the ordering is determined by the default string ordering of the letters that follow. For example, `106A < 106B`. After implementing, you should be able to see:

```Python
>>> cs106a = StanfordCourse("CS", "106A", "Programming Methodology")
>>> cs106b = StanfordCSCourse("CS", "106B", "Programming Abstractions")
>>> cs107 = StanfordCSCourse("CS", "107", "Computer Organzation and Systems")
>>> cs110 = StanfordCSCourse("CS", "110", "Principles of Computer Systems")
>>> cs110 > cs106b
True
>>> cs107 > cs110
False
```

To accomplish this, you will need to implement a magic method `__le__` that will add functionality to determine if a course is a prerequisite for another course. Read up on [total ordering](https://docs.python.org/3/library/functools.html#functools.total_ordering) to figure out what `__le__` should return based on the argument you pass in.

To give a few hints on how to add this piece of functionality might be implemented, consider how you might extract the actual `int` number from the course code attribute.

Additionally, you should implement a `__eq__` on `StanfordCourse`s. Two classes are equivalent if they are in the same department and have the same course code: the course title doesn't matter here.

In [21]:
class StanfordCourse:
    def __init__(self, department, code, title):
        self.department = department
        self.code = code
        self.title = title
        
    def mark_attendance(*students):
        pass
    
    def __le__(self, other):
        self_code_num, other_code_num = int(''.join(filter(lambda x: x.isdigit(), list(self.code)))), int(''.join(filter(lambda x: x.isdigit(), list(other.code))))
        self_code_letter, other_code_letter = ''.join(filter(lambda x: x.isalpha(), list(self.code))), ''.join(filter(lambda x: x.isalpha(), list(other.code)))
        if self_code_num == other_code_num:
            return self_code_letter <= other_code_letter
        return self_code_num <= other_code_num
    
    def __eq__(self, other):
        return self.department == other.department and self.code == other.code

In [23]:
course1 = StanfordCourse('pe', '101', 'Beginner PE')
course2 = StanfordCourse('pe', '102a', 'Intermediate PE')
course3 = StanfordCourse('pe', '102b', 'Intermediate PE 2')
course4 = StanfordCourse('pe', '102b', 'Intermediate PE 2')

print(course1 <= course2) # should return true
print(course2 <= course1) # should return false
print(course2 <= course3) # should return true
print(course3 <= course2) # should return false
print(course3 <= course4) # should return true
print(course3 == course4) # should return true
print(course3 == course2) # should return false

True
False
True
False
True
True
False


#### Sorting

Now that we've written a `__le__` method and an `__eq__` method, we've implemented everything we need to speak about an "ordering" of `StanfordCourse`s. Using the [`functools.total_ordering` decorator](https://docs.python.org/3/library/functools.html#functools.total_ordering), decorate the class so that all of the comparison methods are implemented. You should be able to run

In [34]:
@total_ordering
class StanfordCourse:
    def __init__(self, department, code, title):
        self.department = department
        self.code = code
        self.title = title
        
    def mark_attendance(*students):
        pass
    
    def __le__(self, other):
        self_code_num, other_code_num = int(''.join(filter(lambda x: x.isdigit(), list(self.code)))), int(''.join(filter(lambda x: x.isdigit(), list(other.code))))
        self_code_letter, other_code_letter = ''.join(filter(lambda x: x.isalpha(), list(self.code))), ''.join(filter(lambda x: x.isalpha(), list(other.code)))
        if self_code_num == other_code_num:
            return self_code_letter <= other_code_letter
        return self_code_num <= other_code_num
    
    def __eq__(self, other):
        return self.department == other.department and self.code == other.code
    
    def __repr__(self):
        return 'StanfordCourse(department = {}, code = {}, title = {})'.format(self.department, self.code, self.title)

class StanfordCSCourse(StanfordCourse):
    def __init__(self, department, code, title, recorded=False):
        super().__init__(department, code, title)
        self.is_recorded = recorded

In [36]:
# Let's make CS106A a CS course
cs106a = StanfordCSCourse("CS", "106A", "Programming Methodology")
cs106b = StanfordCSCourse("CS", "106B", "Programming Abstractions")
cs107 = StanfordCSCourse("CS", "107", "Computer Organzation and Systems")
cs110 = StanfordCSCourse("CS", "110", "Principles of Computer Systems")

courses = [cs110, cs106a, cs107, cs106b]
print(courses)
# print(list(map(lambda x: x.code, courses)))
courses.sort()
print(courses)
# print(list(map(lambda x: x.code, courses))) # => [cs106a, cs106b, cs107, cs110]

[StanfordCourse(department = CS, code = 110, title = Principles of Computer Systems), StanfordCourse(department = CS, code = 106A, title = Programming Methodology), StanfordCourse(department = CS, code = 107, title = Computer Organzation and Systems), StanfordCourse(department = CS, code = 106B, title = Programming Abstractions)]
[StanfordCourse(department = CS, code = 106A, title = Programming Methodology), StanfordCourse(department = CS, code = 106B, title = Programming Abstractions), StanfordCourse(department = CS, code = 107, title = Computer Organzation and Systems), StanfordCourse(department = CS, code = 110, title = Principles of Computer Systems)]


### Instructors (optional)

Allow the class to take a splat argument `instructors` that will take any number of strings and store them as a list of instructors.

Modify the way you track attendance in the `StanfordCourse` class to map a Python date object (you can use the `datetime` module) to a data structure tracking what students are there on that day.

In [ ]:
class StanfordCourseWithInstructors:
    pass

### Catalog

Implement a class called `CourseCatalog` that is constructed from a list of `StanfordCourse`s. Write a method for the `CourseCatalog` which returns a list of courses in a given department. Additionally, write a method for `CourseCatalog` that returns all courses that contain a given piece of search text in their title.

Feel free to implement any other interesting methods you'd like.

In [31]:
class CourseCatalog:
    def __init__(self, courses):
        self.courses = courses
       
    def courses_by_department(self, department_name):
        return list(filter(lambda x: x.department == department_name, self.courses))
        
    def courses_by_search_term(self, search_snippet):
        return list(filter(lambda x: search_snippet in x.title, self.courses))

In [37]:
catalog1 = CourseCatalog(courses)
print(catalog1.courses_by_department('pe'))
print(catalog1.courses_by_department('CS'))
print(catalog1.courses_by_search_term('Computer'))

[]
[StanfordCourse(department = CS, code = 106A, title = Programming Methodology), StanfordCourse(department = CS, code = 106B, title = Programming Abstractions), StanfordCourse(department = CS, code = 107, title = Computer Organzation and Systems), StanfordCourse(department = CS, code = 110, title = Principles of Computer Systems)]
[StanfordCourse(department = CS, code = 107, title = Computer Organzation and Systems), StanfordCourse(department = CS, code = 110, title = Principles of Computer Systems)]


## Magic Methods

### Reading

Python provides an enormous number of special methods that a class can override to interoperator with builtin Python operations. You can skim through an [approximate visual list](http://diveintopython3.problemsolving.io/special-method-names.html) from Dive into Python3, or a [more verbose explanation](https://rszalski.github.io/magicmethods/), or the [complete Python documentation](https://docs.python.org/3/reference/datamodel.html#specialnames) on special methods. Fair warning, there are a lot of them, so it's probably better to skim than to really take a deep dive, unless you're loving this stuff.

### Writing (Polynomial Class)

We will write a `Polynomial` class that acts like a number. As a a reminder, a [polynomial](https://en.wikipedia.org/wiki/Polynomial) is a mathematical object that looks like $1 + x + x^2$ or $4 - 10x + x^3$ or $-4 - 2x^{10}$. A mathematical polynomial can be evaluated at a given value of $x$. For example, if $f(x) = 1 + x + x^2$, then $f(5) = 1 + 5 + 5^2 = 1 + 5 + 25 = 31$.

Polynomials are also added componentwise: If $f(x) = 1 + 4x + 4x^3$ and $g(x) = 2 + 3x^2 + 5x^3$, then $(f + g)(x) = (1 + 2) + 4x + 3x^2 + (4 + 5)x^3 = 3 + 4 + 3x^2 + 9x^3$.

Construct a polynomial with a variadic list of coefficients: the zeroth argument is the coordinate of the $x^0$'s place, the first argument is the coordinate of the $x^1$'s place, and so on. For example, `f = Polynomial(1, 3, 5)` should construct a `Polynomial` representing $1 + 3x + 5x^2$.

You will need to override the addition special method (`__add__`) and the callable special method (`__call__`).

You should be able to emulate the following code:

```Python
f = Polynomial(1, 5, 10)
g = Polynomial(1, 3, 5)

print(f(5))  # => Invokes `f.__call__(5)`
print(g(2))  # => Invokes `g.__call__(2)`

h = f + g    # => Invokes `f.__add__(g)`
print(h(3))  # => Invokes `h.__call__(3)`
```

Lastly, implement a method to convert a `Polynomial` to an informal string representation. For example, the polynomial `Polynomial(1, 3, 5)` should be represented by the string `"1 * x^0 + 3 * x^1 + 5 * x^2"`.

In [46]:
class Polynomial:
    def __init__(self, *coefs):
        self.coefs = coefs
    
    def __call__(self, x):
        """Implement `self(x)`."""
        return sum([coef * x**i for i, coef in enumerate(self.coefs)])
    
    def __add__(self, other):
        """Implement `self + other`."""
        return Polynomial(*[s + o for s, o in zip(self.coefs, other.coefs)])
    
    def __str__(self):
        """Implement `str(x)`."""
        return ' + '.join(['{}x^{}'.format(coef, i) for i, coef in enumerate(self.coefs)])

In [47]:
f = Polynomial(1, 5, 10)
print(str(f))
g = Polynomial(1, 3, 5)
print(str(g))

print(f(5))  # => Invokes `f.__call__(5)`
print(g(2))  # => Invokes `g.__call__(2)`

h = f + g    # => Invokes `f.__add__(g)`
print(h(3))  # => Invokes `h.__call__(3)`
print(str(h))

1x^0 + 5x^1 + 10x^2
1x^0 + 3x^1 + 5x^2
276
27
161
2x^0 + 8x^1 + 15x^2


#### Polynomial Extensions (optional)

If you are looking for more, implement additional operations on our `Polynomial` class. You may want to implement `__sub__`, `__mul__`, and `__div__`.

You can also implement more complicated mathematical operations, such as `f.derivative()`, which returns a new function that is the derivative of `f`, or `.zeros()`, which returns a collection of the function's zeros.

If you need even more, write a `classmethod` to construct a polynomial from a string representation of it. You should be able to write:

```
f = Polynomial.parse("1 * x^0 + 3 * x^1 + 5 * x^2")
```

## Done Early?

Take a deep breath. Whatever you're working on, you can do it!

## Credits
Most of this lab was written by @sredmond with modifications by @coopermj.

> With &#129412; by @psarin and @coopermj